# Basketball challenge

In [1]:
from PIL import Image
import torch
import torchvision
from Dataset import Basketball
from Dataprocess import Preprocess
%load_ext autoreload

# Image transformation

In [2]:
img_transformation = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64,48)),
    torchvision.transforms.CenterCrop((48,48)),
    torchvision.transforms.ToTensor()
])


# Preprocessing Dataset

In [3]:
dataprocess = Preprocess().background_subtractor

# Loading Dataset

In [4]:
path = "data"
training_data = Basketball(path, split='training', num_frame = 100, img_transform = img_transformation, dataprocess=dataprocess)

# Display Video

In [ ]:
view1 = training_data.__getitem__(0)[1]

In [ ]:
from IPython import display
import torchvision.transforms.functional as F

for index in range(1):
    view1 = training_data.__getitem__(index)[0]
    for idx, img in enumerate(view1):
        img1 = F.to_pil_image(img)
        display.display(img1.resize((320,240)), Image.NEAREST)
        display.clear_output(wait=True)

# Linear Model

In [5]:
variables = training_data[0][0].numel()

In [6]:
model = torch.nn.Linear(variables , 2)
optimizer = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.4, nesterov=True)
loss_fn = torch.nn.CrossEntropyLoss().cuda()
model = model.cuda()

In [31]:
model.train()
for i in range(0,100):
    samples = training_data[i]
    sample = samples[0].view(1,-1)
    sample = sample.cuda()
    target = torch.tensor([samples[2]])
    target = target.cuda() 
    
    output = model(sample)
    optimizer.zero_grad()
    loss = loss_fn(output, target)
    loss.backward()
    print(i,' ',loss)   

0   tensor(0.6861, device='cuda:0', grad_fn=<NllLossBackward>)
1   tensor(0.6973, device='cuda:0', grad_fn=<NllLossBackward>)
2   tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward>)
3   tensor(0.6973, device='cuda:0', grad_fn=<NllLossBackward>)
4   tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward>)
5   tensor(0.7010, device='cuda:0', grad_fn=<NllLossBackward>)
6   tensor(0.6997, device='cuda:0', grad_fn=<NllLossBackward>)
7   tensor(0.7002, device='cuda:0', grad_fn=<NllLossBackward>)
8   tensor(0.7036, device='cuda:0', grad_fn=<NllLossBackward>)
9   tensor(0.6913, device='cuda:0', grad_fn=<NllLossBackward>)
10   tensor(0.7111, device='cuda:0', grad_fn=<NllLossBackward>)
11   tensor(0.7109, device='cuda:0', grad_fn=<NllLossBackward>)
12   tensor(0.7020, device='cuda:0', grad_fn=<NllLossBackward>)
13   tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward>)
14   tensor(0.7037, device='cuda:0', grad_fn=<NllLossBackward>)
15   tensor(0.6936, device='cuda:0', grad_fn=<NllL

In [33]:
model.eval()

eval_loss = 0
correct = 0
with torch.no_grad():
    for i in range(100,135):
        samples = training_data[i]
        sample = samples[0].view(1,-1)
        sample = sample.cuda()
        target = torch.tensor([samples[2]])
        target = target.cuda() 
        output = model(sample)

        eval_loss += loss_fn(output, target)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
eval_loss /= 35
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    eval_loss, correct, 35,
    100. * correct / 35))


Test set: Average loss: 0.6919, Accuracy: 19/35 (54%)

